### Install requirements

In [ ]:
!pip install sagemaker --quiet --upgrade --force-reinstall

### JumpStart example

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

In [ ]:
model_jumpstart = JumpStartModel(model_id="huggingface-llm-falcon-7b-bf16")

In [ ]:
predictor_jumpstart = model_jumpstart.deploy(instance_type="ml.g5.2xlarge")

In [ ]:
prompt = "What are the places to see in Rome?"

payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.4,
        "max_new_tokens": 256,
        "stop": ["<|endoftext|>", "</s>"],
    },
}

response = predictor_jumpstart.predict(payload)
print(response[0]["generated_text"])

##### Deleting endpoint

In [ ]:
predictor_jumpstart.delete_model()
predictor_jumpstart.delete_endpoint()

### HuggingFace Estimator Example

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()

In [ ]:
hub = {
    'HF_MODEL_ID':'tiiuae/falcon-7b',
    'HF_TASK':'text-generation'
}
model_huggingface = HuggingFaceModel( transformers_version='4.6', pytorch_version='1.7', py_version='py36', env=hub,role=aws_role)

In [ ]:
predictor_huggingface = model_huggingface.deploy(initial_instance_count=1, instance_type="ml.g5.4xlarge")

##### Deleting endpoint

In [ ]:
predictor_huggingface.delete_model()
predictor_huggingface.delete_endpoint()

### SageMaker Custom Docker image example

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models
from sagemaker.predictor import Predictor
from sagemaker.model import Model
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id="huggingface-llm-falcon-7b-bf16",
    model_version="*",
    instance_type="ml.g5.2xlarge",
)

model_uri = model_uris.retrieve(
    model_id="huggingface-llm-falcon-7b-bf16", model_version="*", model_scope="inference"
)

In [ ]:
deploy_image_uri

In [ ]:
model_uri

In [ ]:
model_custom = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    env={
        "SAGEMAKER_MODEL_SERVER_WORKERS": "1",
        "TS_DEFAULT_WORKERS_PER_MODEL": "1",
        "HF_MODEL_ID":"tiiuae/falcon-7b",
        "HF_TASK":"text-generation"
    }
)

In [ ]:
predictor_custom = model_custom.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge",
    predictor_cls=Predictor,
)

##### Deleting endpoint

In [ ]:
predictor_custom.delete_model()
predictor_custom.delete_endpoint()